### Notebook for retrieving average roof quality per neighboorhood

In [ ]:
import numpy as np
import pandas as pd
import requests

In [ ]:
data = pd.read_csv('../data/groningen/pc6hnr20170801_gwb.csv', sep=';')

In [ ]:
# Only take data from Groningen
data = data[data['Gem2017'] == 14]
data = data.drop('Gem2017', axis=1)

In [ ]:
data = data.groupby("Buurt2017").head(30)
data = np.asarray(data)

##### First retrieve the coordinate data from argcis given a ps6 + number

In [ ]:
from tqdm import tqdm
from time import sleep
info = []
for entry in tqdm(data):
    url = "https://basisregistraties.arcgisonline.nl/arcgis/rest/services/BAG/BAGv2/MapServer/0/query?where=postcode%3D%27" + entry[0]  + "%27+AND+huisnummer%3D%27" + str(entry[1]) + "%27&text=&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=3857&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson"
    contents = requests.get(url).json()
    sleep(0.05)
    try:
        d = contents['features'][0]['geometry']
        long = d['x']
        lat = d['y']
        info.append([entry[0], entry[1], entry[2], long, lat])
    except:
        continue

##### Then retrieve the item id zonatlas uses to identify the building on that coordinate

In [ ]:
out = []
for row in info[:]:
    url = "http://detailshera.zonatlas.nl/groningen/query?olon={}&olat={}&active_topic=pv".format(row[3], row[4])
    contents = requests.get(url)
    try:
        d = contents.json()['item_id']
        r = list(row)
        r.append(d)
        out.append(r)
    except:
        continue

##### Then retrieve the roof quality on these adresses per building

In [ ]:
done = []
for row in out:
    url = 'http://detailshera.zonatlas.nl/groningen/query/building/{}?pv=1&_=15273287'.format(row[-1])
    contents = requests.get(url)
    k = None
    if("t\">Zeer geschikt" in contents.text):
        k = 2
    elif("t\">Geschikt" in contents.text):
        k = 1
    else:
        k = 0
    r = list(row)
    r.append(k)
    done.append(r)

##### Aggegrate the data per neighboorhood and save to csv

In [ ]:
df = pd.DataFrame(done, columns=['postalcode', 'number', 'buurt', 'long', 'lat', 'obj_id', 'score'])
df[['long', 'lat']] = df[['long', 'lat']].apply(pd.to_numeric)
df.groupby('buurt').mean()[['long', 'lat']].to_csv('longlatbuurt.csv')
df.to_csv('longlatperbuurt.csv')